# Cosmic Ray Ionization and Heating

Following Stacy & Bromm (2007), we calculate the rates for ionization ($k_{\rm \small CR}$) and heating ($\Gamma_{\rm \small CR}$) of neutral hydrogen by cosmic rays as follows:
    $$\Gamma_{\rm \small CR} = n_{\rm \small H} E_{\rm \small heat} k_{\rm \small CR}$$
and
    $$
    k_{\rm \small CR} = \frac{1.82\times10^{-7}\,{\rm \small eV\,s}^{-1}}{50\,{\rm \small eV}} 
    \int_{\epsilon_{\rm min}}^{\epsilon_{\rm max}} f(\epsilon) \frac{dn_{\rm \tiny CR}}{d\epsilon} d\epsilon,
    $$
where
    $$f(\epsilon) = (1 + 0.0185 \,{\rm ln}\beta )\, \frac{2 \beta^2}{\beta_0^3 + 2 \beta^3}$$
and
    $$\beta =  \sqrt{1 - \left( \frac{\epsilon}{m_{\rm \tiny H}c^2}+1 \right)^{-2}},$$
and the differential CR energy spectrum is given by
    $$\frac{dn_{\rm \tiny CR}}{d\epsilon} = \frac{u_{\rm \small CR} \epsilon^{-2}}{{\rm ln}\,\epsilon_{\rm max}{\large /}\epsilon_{\rm min}}.$$

Here, $n_{\rm \small H}$ is the number density of hydrogen, $E_{\rm \small heat}$ is the energy deposited as heat in a neutral medium, $m_{\rm \small H}$ is the mass of hydrogen, $c$ is the speed of light, $\beta = v/c,$ and $\beta_0$ is the cutoff below which the interaction between CRs and the gas decreases sharply.

**Note that we have assumed that the gas in question remains optically thin, such that attenuation of the incident CR flux is negligible.**

In [ ]:
#First, define beta.
def xbeta(epsilon):
    mHc2 = 938272046 #rest energy of a proton in eV
    return np.sqrt(1 - (epsilon/mHc2 + 1)**(-2))

In [ ]:
#Define f(epsilon).
def f(epsilon):
    B0 = 0.01
    B = xbeta(epsilon)
    return (1 + 0.0185 * np.log(B)) * 2*B**2 / (B0**3 + 2*B**3)

In [ ]:
#Define differential energy spectrum.
def dnCR(epsilon, ucr, emin, emax):
    return ucr / epsilon / epsilon / np.log(emax/emin)

In [ ]:
def integrand(epsilon, emin, emax):
    return f(epsilon) / epsilon / epsilon / np.log(emax/emin)

#### Check to see that what we get matches Athena's parameterization.

First, do the integral over epsilon, from $10^6$eV to $10^{15}$eV.  We have to do it by hand because scipy's built-in integration routines all use quadrature, which doesn't work for a power law over many orders of magnitude.

In [ ]:
%matplotlib inline
import os
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
sns.set_context('poster')

In [ ]:
emin = 1e6
emax = 1e15
n_steps = 1000
integral = 0.0

logmin = np.log10(emin)
logmax = np.log10(emax)
for i in xrange(n_steps):
    epsilon = (logmax - logmin) / n_steps * (i + 0.5) + logmin
    e_start = (logmax - logmin) / n_steps* (i) + logmin
    e_stop = (logmax - logmin) / n_steps * (i + 1.0) + logmin
    epsilon = 10**epsilon
    e_start = 10**e_start
    e_stop = 10**e_stop
    De = e_stop - e_start
    #print epsilon, f(epsilon), De, integrand(epsilon, emin, emax)
    integral += integrand(epsilon, emin, emax) * De
integral = integral * 1.82e-7 / 50
print integral

Now, Athena's parameterization uses $U_{\rm \small CR} = 2\times10^{-15} {\rm erg\,cm^{-3}}$, $E_{\rm \small heat} = 6{\rm eV}$, and $n_{\rm \small H} = 1{\rm cm^{-3}}.$

In [ ]:
u_athena = 2e-15 * 6.24150934e11 # convert from erg to eV
eHeat =  6 / 6.24150934e11
ion_rate = u_athena * integral
heat_rate = ion_rate * eHeat

Plugging those values in and making sure to correctly convert from erg to eV, we get the following for $k_{\rm \small CR}$ and $\Gamma_{\rm \small CR}$:

In [ ]:
print ion_rate

In [ ]:
print heat_rate

These values are off from Athena's parameterizations by about 2x.  I haven't figured out exactly what accounts for the discrepancy yet, but it's close enough to start a simulation.

# Complete H/He Ionization and Heating

From Schlickeiser (2002; eq. 5.3.42), the full-blown prescription for the kinetic energy loss rate of a CR ion traveling through a neutral medium is given by
$$
-\frac{{\rm d}\epsilon}{{\rm d}t} = 
\frac{3 c \sigma_{\rm \tiny T} Z^2 (m_e c^2)}{4\beta}
\sum_s n_s [B_s + B'(\alpha Z / \beta)],
$$
where (eq. 5.3.43)
$$
B_s = {\rm ln} \left[ \frac{2m_e c^2 \beta^2 Q_{\rm \small max}}{I_s^2 (1-\beta^2)} \right] - 2\beta^2 - \frac{2C_s}{z_s} - \delta_s
$$
and (eq. 5.3.44)
$$
Q_{\rm \small max} \simeq
\frac{2m_e c^2 \beta^2 \gamma^2} {1 + \frac{2m_e \epsilon}{m^2 c^2}}
$$
    
+ $c \equiv$ speed of light
+ $\alpha \equiv$ fine structure constant
+ $\sigma_{\rm \tiny T} \equiv$ Thompson cross-section
+ $\epsilon \equiv$ cosmic ray energy
+ $Z \equiv$ cosmic ray charge
+ $M \equiv$ cosmic ray mass
+ $m_p \equiv$ proton mass
+ $m_e \equiv$ electron mass
+ $n_s \equiv$ number density of species $s$ in medium
+ $z_s \equiv$ charge of species $s$ 
+ $I_s \equiv$ average of all ionization and excitation potentials of species $s$; $\left[ I_{\rm H} = 19{\rm eV}, I_{\rm He} = 44{\rm ev} \right]$
+ $\beta \equiv$ cosmic ray velocity; $\left[ \beta = v/c = \sqrt{1 - \left( \frac{\epsilon}{M c^2}+1 \right)^{-2}} \right]$
+ $\gamma \equiv$ cosmic ray Lorentz factor?


**We will restrict ourselves to high-energy CR protons, in which case $Z=1$, $M=m_p$ and $B' \rightarrow 0$.**

For $\epsilon \leq 918 m_p c^2$, the asymptotic behavior of $Q_{\rm \small max}$ is given by
$$
Q_{\rm \small max} = 2 m_e (c \beta \gamma)^2
$$

According to Schlickeiser, using this approximation and $Z=1$, eq. 5.3.42 should reduce to
$$
-\frac{{\rm d}\epsilon}{{\rm d}t} = 
\frac{3 c \sigma_{\rm \tiny T}(m_e c^2)}{2}
\sum_s n_s \left( {\rm ln}\frac{2m_e c^2}{I_s} + {\rm ln}\beta + \frac{\beta^4}{2} \right)
$$
with 
$$
-\frac{{\rm d}\epsilon}{{\rm d}t} = 1.82\times10^{-7}[n_{\rm HI} + 2 n_{\rm H_2}] (1 + 0.0185 \,{\rm ln}\beta )\, \frac{2 \beta^2}{\beta_0^3 + 2 \beta^3}
$$
providing a useful interpolation formula (eq. 5.3.51).

**_Let's test this._**

In [ ]:
from __future__ import division
from astropy import units as u
from astropy import constants as const
from scipy import constants as physical_constants
import scipy as sp

In [ ]:
c = const.c.cgs
c2 = c*c
sigmaT = (physical_constants.value('Thomson cross section') * u.meter**2).to('cm2')
me = const.m_e.cgs
mp = const.m_p.cgs
ee = me*c2
ee = ee.to('eV')
I_H = 19 * u.eV
I_He = 44 * u.eV
n_H = 1.14 / u.centimeter**3
n_He = 0.00013 / u.centimeter**3

In [ ]:
#Re-define beta with units.
def beta(epsilon):
    mHc2 = 938272046*u.eV #rest energy of a proton in eV
    return np.sqrt(1 - (epsilon/mHc2 + 1)**(-2))

In [ ]:
def part(ep, n, I):
    return n * (np.log(2*ee/I) + np.log(beta(ep)) + 0.5*(beta(ep))**4)
    
def D1(epsilon):
    tot = part(epsilon, n_H, I_H) + part(epsilon, n_He, I_He)
    return 1.5 * c * sigmaT * ee * tot

In [ ]:
E = np.logspace(-3,12,100) * u.eV
out1 = D1(E)
out2 = 1.82e-7 * n_H.value * f(E.value)

In [ ]:
plt.loglog(E.value/1e6, out1.value)
plt.loglog(E.value/1e6, out2)
plt.xlabel('CR proton energy [MeV]')
plt.ylabel('d$\epsilon$/d$t$ [eV s$^{-1}$]')
plt.xlim(1e-3, 1e6)
plt.ylim(1e-9, 1e-3)

*_Clearly that didn't work._* In retrospect it is now clear that the asymptotic behavior referred to does not cover the entire range we are interested in ($10^6$ to $10^{15}$ eV).

### Using full prescription for $Q_{\rm max}$:

Going back to equations 5.3.42 - 5.3.44, we have
$$
-\frac{{\rm d}\epsilon}{{\rm d}t} = 
\frac{3 c \sigma_{\rm \tiny T} Z^2 (m_e c^2)}{4\beta}
\sum_s n_s [B_s + B'(\alpha Z / \beta)],
$$
$$
B_s = {\rm ln} \left[ \frac{2m_e c^2 \beta^2 Q_{\rm \small max}}{I_s^2 (1-\beta^2)} \right] - 2\beta^2 - \frac{2C_s}{z_s} - \delta_s,
$$
and
$$
Q_{\rm \small max} = 
\frac{\epsilon + 2M c^2} {1 + \frac{(M + m_e)^2 c^2}{2m_e \epsilon}}
$$

As before, we can assume $Z=1$, $M=m_p$ and $B' \rightarrow 0$.  In addition, following the implicit logic of eq. 5.3.50, we will assume $C_s \rightarrow 0$ and $\delta_s \rightarrow 0$.  Using these assumptions, and noting that $m_p + m_e \simeq m_p$,
$$
-\frac{{\rm d}\epsilon}{{\rm d}t} = 
\frac{3 c \sigma_{\rm \tiny T} Z^2 (m_e c^2)}{4\beta}
\sum_s n_s B_s,
$$
$$
B_s = {\rm ln} \left[ \frac{2m_e c^2 \beta^2 Q_{\rm \small max}}{I_s^2 (1-\beta^2)} \right] - 2\beta^2,
$$
and
$$
Q_{\rm \small max} \simeq
\frac{2m_e c^2 \beta^2 \gamma^2} {1 + \frac{2m_e \epsilon}{m_p^2 c^2}}
$$

In [ ]:
def gamma(ep):
    return 1 / np.sqrt(1 - beta(ep)**2)

def Bs(ep, I):
    numerator = (2*ee * beta(ep)**2 * gamma(ep)**2)**2 
    denominator = I*I * (1 + (2*me*ep)/(mp*c)**2)
    return np.log(numerator/denominator) - 2*beta(ep)**2

def Bprime(x):
    digamma = sp.special.psi(1 + x.value*1j)
    return .5 * (sp.special.psi(1) - digamma.real)
    
def fullD(epsilon):
    sigma = n_H * Bs(epsilon, I_H) + n_He * Bs(epsilon, I_He)
    return 0.75 * c * sigmaT * ee * sigma / beta(epsilon)

In [ ]:
E = np.logspace(-3,12,100) * u.eV
schlickeiser = fullD(E)
asym = D1(E)
interp = 1.82e-7 * n_H.value * f(E.value)

In [ ]:
f515 = np.loadtxt(os.getenv('HOME')+'/data/otherPeoplesWork/Schlickeiser2002/data.csv', delimiter=',')

In [ ]:
plt.loglog(E.value/1e6, schlickeiser.value, lw=2, label='full solution')
plt.loglog(E.value/1e6, asym.value, lw=2, label ='high-$\epsilon$ asymptotic solution')
plt.loglog(E.value/1e6, interp, lw=2, label='combined interpolation (Athena)')
plt.loglog(f515[:,0], f515[:,1], 'k--', lw=3, label='Figure 5.15 from Schlickeiser')
plt.xlabel('CR proton energy [MeV]')
plt.ylabel('d$\epsilon$/d$t$ [eV s$^{-1}$]')
plt.xlim(1e-3, 1e6)
plt.ylim(1e-9, 1e-3)
plt.legend()
plt.savefig('figures/dEdt.png', bbox_inches='tight')

###### Still didn't quite work. 

I'm beginning to suspect that $C_s \rightarrow 0$ and $\delta_s \rightarrow 0$ is not a valid assumption.
But first, lets check the veracity of 5.3.44. Namely, let's check that 
$$
Q_{\rm \small max} = 
\frac{\epsilon + 2M c^2} {1 + \frac{(M + m_e)^2 c^2}{2m_e \epsilon}}
 \simeq
\frac{2m_e c^2 \beta^2 \gamma^2} {1 + \frac{2m_e \epsilon}{m_p^2 c^2}}.
$$
Again, we assume $M=m_p$:

In [ ]:
def Qmax(epsilon):
    numerator = epsilon + 2 * mp * c2
    denominator = 1 + (mp + me)**2 * c2 / (2*me * epsilon)
    return numerator / denominator

def Qapprox(epsilon):
    denom = 1 + (2 * me * epsilon / (mp * mp * c2))
    return 2 * ee * beta(epsilon)**2 * gamma(epsilon)**2 / denom

In [ ]:
q1 = Qmax(E)
q2 = Qapprox(E)

plt.loglog(E.value/1e6, q1.value, 'k', lw=2, label='full Q')
plt.loglog(E.value/1e6, q2.value, 'r--', lw=2, label='approximate Q')

plt.legend(loc=0)

In [ ]:
plt.semilogx(E.value, q1.value/q2.value)
plt.ylim(.99,1.01)

# A different approach: following Jasche, Ciardi & En$\beta$lin (2007)

The approach detailed in Schlickeiser (2002) is incomplete, referring the reader to Sternheimer (1952) for a complete explanation of the $C_s$ and $\delta_s$ terms in equation 5.3.43.  Since Jasche provides a comprehensive description of their ionization and heating machinery, we'll follow them instead.  Unfortunately, this will require grokking a new set of notations...

Jasche et al. describe things in terms of a dimensionless momentum, $p$.  This is given by $p=p_{\rm \tiny CR}/m_p c$, where $p_{\rm \tiny CR}$ is the particle's momentum.  However, I still want to have things described in terms of energy at the end of the day, so we'll take advantage of the fact that $p_{\rm \tiny CR} = m_p c \beta \gamma$ and $\epsilon = m_p c^2 \gamma$ to define $p$ in terms of epsilon: $$p = \frac{\beta \epsilon}{m_p c^2}$$

In [ ]:
def p(epsilon): 
    return beta(epsilon) * gamma(epsilon) #epsilon.to('erg') / mp / c2

We'll need to introduce some other new notation as well:

+ $e \equiv$ charge of an electron
+ $n_z \equiv$ number density of atomic species with electron number $Z$
+ $I_z \equiv$ ionization potential of species $Z$
+ $L_0 \equiv$ the stopping number (Ziegler 1999)
+ $L_{\rm Bloch} \equiv$ Bloch correction for slow particles (Mannheim & Schlickeiser 1994)
+ $b(\gamma) = \sqrt{1 + 2 \gamma m_e / m_p + (m_e/m_p)^2} \equiv$ correction factor accounting for maximum kinetic energy that can be imparted to a free electron in a given collision
+ $\delta_z \equiv$ density correction factor accounting for the screen effect of a medium as it becomes polarized by the CRs moving through it

With this updated notation, we can describe the ionization losses of the CR using the Bethe-Bloch equation:
$$ -\left[\frac{{\rm d}\epsilon}{{\rm d}t}\right]_{\rm ion} = \frac{4\pi e^4}{m_e \beta c} \sum_z Z n_z [L_0 + L_{\rm Bloch}]$$
where $$L_0 = {\rm ln}\left(\frac{2 m_e c^2 p^2}{I_z b(\gamma)} \right) -\beta^2 - \frac{\delta_z}{2}$$
and $$L_{\rm Bloch} = \frac{1}{2} \left[ \Psi(1) - {\rm Re}\Psi\left(1 + \frac{i\alpha}{\beta} \right) \right]$$
such that the full equation can be written as 
$$ = \frac{4\pi e^4}{m_e \beta c} \sum_z Z n_z \left[ {\rm ln}\left(\frac{2 m_e c^2 p^2}{I_z b(\gamma)} \right) -\beta^2 - \frac{\delta_z}{2} + \frac{\Psi(1)}{2} -  \frac{1}{2}{\rm Re}\Psi\left(1 + \frac{i\alpha}{\beta} \right) \right]$$
where 



In [ ]:
qe = const.e.gauss


In [ ]:
mp, me, qe

In [ ]:
def beta(epsilon):
    mHc2 = 938272046*u.eV #rest energy of a proton in eV
    return np.sqrt(1 - (epsilon/mHc2 + 1)**(-2))

def gamma(ep):
    return 1 / np.sqrt(1 - beta(ep)**2)

def bgamma(gamma):
    return np.sqrt(1 + 2*gamma*me/mp + (me/mp)**2)

def deltaz(ep, Iz, y0, y1, az, kz):
    y = np.log(p(ep))
    #wpl = np.

def L0(ep, I):
    top = 2 * me * c2 * p(ep)**2
    bot = I.to('erg') * bgamma(gamma(ep))
    return np.log(top/bot) - beta(ep)**2 #!!!!! Ignoring delta_z for the moment...

def Lb(ep):
    a = sp.special.psi(1)
    b = 1 +(1j/137/beta(ep))
    b = sp.special.psi(b.value)
    return 0.5 * (a+np.real(b))

def dTdt(ep):
    sigma = n_H * (L0(ep, 13.6*u.eV) + Lb(ep)) + 2 * n_He * (L0(ep, 24.6*u.eV) + Lb(ep))
    x = 4 * np.pi * qe**4 * sigma / (me * beta(ep) * c)
    return x.cgs.to('eV / s')

In [ ]:
def dedt(epsilon):
    sigma = n_H * Bs(epsilon, I_H) + n_He * Bs(epsilon, I_He)
    return 0.75 * c * sigmaT * ee * sigma / beta(epsilon)

In [ ]:
E = np.logspace(-3,12,1000) * u.eV
jasche = dTdt(E)
schlickeiser = dedt(E)
interp = 1.82e-7 * n_H.value * f(E.value)
jasche.unit

In [ ]:
plt.loglog(E.value/1e6, schlickeiser.value, lw=2, label='Schlickeiser (2002)')
plt.loglog(E.value/1e6, jasche.value, lw=2, label ='Jasche et al. (2007)')
plt.loglog(E.value/1e6, interp, lw=2, label='combined interpolation (Athena)')
plt.loglog(f515[:,0], f515[:,1], 'k--', lw=3, label='Figure 5.15 from Schlickeiser')
plt.xlabel('CR proton energy [MeV]')
plt.ylabel('d$\epsilon$/d$t$ [eV s$^{-1}$]')
plt.xlim(1e-3, 1e6)
plt.ylim(1e-9, 1e-3)
plt.legend()
#plt.savefig('figures/dEdt.png', bbox_inches='tight')

In [ ]:
jasche.unit

In [ ]:
hbar = const.hbar
hbar

In [ ]:
c

In [ ]:
const.e.gauss**2 / const.hbar.cgs / const.c.cgs

In [ ]:
1.60218e-19**2 / 1.05257e-34 / 2.9979e8

In [ ]:
const.e.gauss

In [ ]:
u.erg.find_equivalent_units()